In [ ]:
import numpy as np
from bikewheelcalc import BicycleWheel, Rim, Hub, Spoke
import bikewheellib as bl
from IPython.display import display

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

%matplotlib inline

In [ ]:
rad_only = [True, False, False]
wheels = [BicycleWheel() for i in range(3)]

for i in range(len(wheels)):
    wheels[i].hub = Hub(diam1=0.05, width1=0.025, width2=0.025)

    R = 0.300
    GJ = 25.0
    EI1 = 100
    EI2 = GJ / 0.2

    wheels[i].rim = Rim(radius=R, area=100e-6,
                        I11=GJ / 26.0e9, I22=EI2 / 69.0e9, I33=EI1 / 69.0e9, Iw=0.0 / 69.0e9,
                        young_mod=69.0e9, shear_mod=26.0e9)

wheels[0].lace_radial(n_spokes=36, diameter=1.8e-3, young_mod=200e9, offset=0.0)
wheels[0].apply_tension(800)

wheels[1].lace_cross(n_spokes=36, n_cross=1, diameter=1.8e-3, young_mod=200e9, offset=0.0)
wheels[1].apply_tension(800)

wheels[2].lace_cross(n_spokes=36, n_cross=3, diameter=1.8e-3, young_mod=200e9, offset=0.0)
wheels[2].apply_tension(800)

In [ ]:
fig = plt.figure(figsize=(6.5, 5.0))

gs = GridSpec(len(wheels), 4, width_ratios=[1, 2.5, 2.5, 2.5])

draw_opts = {'spk_ls_width': 0.5, 'spk_ds_width': 0.5, 'rim_width': 1.0}

# Spoke numbers
s_num = np.arange(36)

for i in range(len(wheels)):

    rr = bl.ModeMatrix(wheels[i], N=36)
    K = rr.calc_K_rim(buckling=False) + rr.calc_K_spk(smeared_spokes=True)

    # Draw wheel
    wheels[i].draw(plt.subplot(gs[i, 0]), draw_opts)
    
    # Calculate change in spoke tension for radial load
    F_ext = rr.calc_F_ext(np.array([np.pi]),
                      np.array([[0., 1., 0., 0.]]))
    d = np.linalg.solve(K, F_ext)

    delta_T = [-s.EA/s.length * s.n.dot(rr.B_theta(s.rim_pt[1], [0, 1, 2]).dot(d))
               for s in wheels[i].spokes]

    plt.subplot(gs[i, 1]).bar(s_num, delta_T, width=0.6, color='#1F77B4', edgecolor='none')
    plt.subplot(gs[i, 1]).bar(18, delta_T[18], width=0.6, color='#D62728', edgecolor='none')


    if not rad_only[i]:
        # Calculate change in spoke tension for unit tangential load
        F_ext = rr.calc_F_ext(np.array([np.pi]),
                          np.array([[0., 0., 1., 0.]]))
        d = np.linalg.solve(K, F_ext)

        delta_T = [-s.EA/s.length * s.n.dot(rr.B_theta(s.rim_pt[1], [0, 1, 2]).dot(d))
                   for s in wheels[i].spokes]

        plt.subplot(gs[i, 2]).bar(s_num, delta_T, width=0.6, color='#1F77B4', edgecolor='none')
        plt.subplot(gs[i, 2]).bar(18, delta_T[18], width=0.6, color='#D62728', edgecolor='none')

        # Calculate change in spoke tension for a combined loading case
        F_ext = rr.calc_F_ext(np.array([np.pi]),
                          np.array([[0., 500., 50., 0.]]))
        d = np.linalg.solve(K, F_ext)

        delta_T = [-s.EA/s.length * s.n.dot(rr.B_theta(s.rim_pt[1], [0, 1, 2]).dot(d))
                   for s in wheels[i].spokes]

        plt.subplot(gs[i, 3]).bar(s_num, delta_T, width=0.6, color='#1F77B4', edgecolor='none')
        plt.subplot(gs[i, 3]).bar(18, delta_T[18], width=0.6, color='#D62728', edgecolor='none')
    
    for j in range(1, 4):
        plt.subplot(gs[i, j]).set_xlim([-1, 36])
        plt.subplot(gs[i, j]).set_xticks([])

    plt.subplot(gs[i, 1]).set_ylim(-0.5, 0.1)
    plt.subplot(gs[i, 1]).set_yticks([-.5, -.4, -.3, -.2, -.1, 0., 0.1])
    plt.subplot(gs[i, 1]).set_yticklabels(['-0.5', '', '', '', '', '0', ''])
    
    plt.subplot(gs[i, 2]).set_ylim(-0.5, 0.5)
    plt.subplot(gs[i, 2]).set_yticks(np.arange(-0.5, 0.51, 0.1))
    plt.subplot(gs[i, 2]).set_yticklabels(['-0.5', '', '', '', '', '0', '', '', '', '', '0.5'])
    
    plt.subplot(gs[i, 3]).set_ylim(-200, 50)
    plt.subplot(gs[i, 3]).set_yticks([-200, -150, -100, -50, 0, 50])

plt.tight_layout()
plt.savefig('../figs/stress_analysis/_python_rad_tan_Tinf.pdf')